# Shortest Path Visualizer

**Instructions:**
1. Make sure you're using the `routing-pipeline/venv` Python environment
2. Run cells sequentially from top to bottom
3. Adjust SOURCE_EDGE and TARGET_EDGE in the "Example Query" section

**Troubleshooting:**
- If freezing occurs, try restarting the kernel: `Kernel > Restart Kernel`
- Ensure all dependencies are installed in the venv
- Check that dataset paths in config/datasets.yaml are correct

In [1]:
import subprocess
import pandas as pd
import folium
from shapely import wkt
import yaml
from pathlib import Path
import json
import h3

In [2]:
import networkx as nx

In [3]:
# Quick diagnostic - verify environment
import sys
print(f"Python: {sys.version}")
print(f"Executable: {sys.executable}")
print("✓ All imports successful")

Python: 3.10.12 (main, Nov  4 2025, 08:48:33) [GCC 11.4.0]
Executable: /home/kaveh/projects/routing-pipeline/venv/bin/python
✓ All imports successful


## Configuration

Load dataset configuration from `config/datasets.yaml`

In [4]:
# Load dataset configuration
config_path = Path('../config/datasets.yaml')
with open(config_path, 'r') as f:
    config = yaml.safe_load(f)

# Display available datasets
print("Available datasets:")
for idx, dataset in enumerate(config['datasets']):
    print(f"{idx}: {dataset['name']}")

# Select dataset (change this index to select different dataset)
DATASET_INDEX = 1  # 0 for Somerset, 1 for Burnaby
dataset = config['datasets'][DATASET_INDEX].copy()

# Adjust paths to be relative to notebook directory
# Config paths are relative to routing-pipeline/, we're in routing-pipeline/notebooks/
for key in ['shortcuts_path', 'edges_path', 'binary_path']:
    if key in dataset and not Path(dataset[key]).is_absolute():
        # Add one more ../ to go from notebooks/ to routing-pipeline/
        dataset[key] = str(Path('..') / dataset[key])

print(f"\nSelected dataset: {dataset['name']}")
print(f"Shortcuts: {dataset['shortcuts_path']}")
print(f"Edges: {dataset['edges_path']}")
print(f"Binary: {dataset['binary_path']}")

Available datasets:
0: Somerset
1: Burnaby

Selected dataset: Burnaby
Shortcuts: ../../spark-shortest-path/output/Burnaby_shortcuts_final
Edges: ../../osm-to-road-network/data/output/Burnaby_driving_simplified_edges_with_h3.csv
Binary: ../../dijkstra-on-Hierarchy/build/shortcut_router


## Load Edge Data

Load the edge CSV file to get geometry and metadata for visualization

In [5]:
# Load edges CSV
edges_df = pd.read_csv(dataset['edges_path'])
print(f"Loaded {len(edges_df)} edges")
print(f"\nColumns: {edges_df.columns.tolist()}")
print(f"\nSample edge data:")
edges_df.head()

Loaded 34965 edges

Columns: ['length', 'maxspeed', 'geometry', 'highway', 'cost', 'incoming_cell', 'outgoing_cell', 'lca_res', 'id']

Sample edge data:


,length,maxspeed,geometry,highway,cost,incoming_cell,outgoing_cell,lca_res,id
0,98.503,30.0,LINESTRING (-122.92154693603516 49.27764129638...,tertiary_link,11.82036,644733695069033029,644733695069283890,8,0
1,53.933,30.0,LINESTRING (-122.92154693603516 49.27764129638...,tertiary,6.47196,644733695069345412,644733695069283890,9,1
2,684.050,50.0,LINESTRING (-122.92154693603516 49.27764129638...,tertiary,49.25160,644733695063774494,644733695069283890,7,2
3,35.488,30.0,LINESTRING (-122.92096710205078 49.27793121337...,tertiary,4.25856,644733695069337920,644733695069345412,10,3
4,53.933,30.0,LINESTRING (-122.92096710205078 49.27793121337...,tertiary,6.47196,644733695069283890,644733695069345412,9,4


In [6]:
# Create edge lookup dictionary for fast access
edge_lookup = edges_df.set_index('id').to_dict('index')
print(f"Created lookup dictionary for {len(edge_lookup)} edges")

Created lookup dictionary for 34965 edges


## Build NetworkX Graph

Load edge graph and create NetworkX directed graph for comparison

In [7]:
# Load edge graph CSV
district = "Burnaby"  # or "Burnaby"
#district = "Somerset"  # or "Burnaby"

edges_graph_PATH = f"../../osm-to-road-network/data/output/{district}_driving_edge_graph.csv"

print(f"Loading edge graph from: {edges_graph_PATH}")
edges_graph_df = pd.read_csv(edges_graph_PATH)
print(f"Loaded {len(edges_graph_df)} edge connections")
print(f"\nColumns: {edges_graph_df.columns.tolist()}")
print(f"\nSample data:")
edges_graph_df.head()

Loading edge graph from: ../../osm-to-road-network/data/output/Burnaby_driving_edge_graph.csv
Loaded 98901 edge connections

Columns: ['incoming_edge', 'outgoing_edge']

Sample data:


,incoming_edge,outgoing_edge
0,30721,30720
1,20799,1302
2,8066,8061
3,15745,15742
4,23542,23545


In [8]:
# Create NetworkX directed graph
G = nx.DiGraph()

# Create edge costs lookup from edges_df for fast access
edge_costs = {row['id']: row['cost'] for _, row in edges_df.iterrows()}

# Add edges with weights from edge graph
# Direction: incoming_edge → outgoing_edge (path flows from incoming to outgoing)
for _, row in edges_graph_df.iterrows():
    outgoing = row['outgoing_edge']
    incoming = row['incoming_edge']
    
    # Get cost from edge metadata (cost is stored per edge, not per connection)
    # Use the incoming edge's cost as the weight
    cost = edge_costs.get(incoming, 0)
    
    G.add_edge(incoming, outgoing, weight=cost)

print(f"NetworkX Graph Statistics:")
print(f"  Nodes: {G.number_of_nodes()}")
print(f"  Edges: {G.number_of_edges()}")
print(f"  Is directed: {G.is_directed()}")
print(f"\nSample edges (incoming → outgoing):")
for i, (u, v, data) in enumerate(G.edges(data=True)):
    if i >= 5:
        break
    print(f"  {u} → {v} (weight: {data['weight']:.2f})")

NetworkX Graph Statistics:
  Nodes: 34965
  Edges: 98901
  Is directed: True

Sample edges (incoming → outgoing):
  30721 → 30720 (weight: 27.73)
  30721 → 30718 (weight: 27.73)
  30721 → 30719 (weight: 27.73)
  30720 → 1799 (weight: 13.63)
  30720 → 1800 (weight: 13.63)


## NetworkX Shortest Path Comparison

Compare NetworkX's shortest path with your algorithm

In [9]:
def networkx_shortest_path(source_edge, target_edge, graph):
    """
    Compute shortest path using NetworkX for comparison
    
    Args:
        source_edge: Source edge ID
        target_edge: Target edge ID
        graph: NetworkX DiGraph
    
    Returns:
        Dictionary with distance and path
    """
    try:
        # Compute shortest path
        path = nx.shortest_path(graph, source=source_edge, target=target_edge, weight='weight')
        distance = nx.shortest_path_length(graph, source=source_edge, target=target_edge, weight='weight')
        
        return {
            'distance': distance,
            'path': path
        }
    except nx.NetworkXNoPath:
        print(f"No path found from {source_edge} to {target_edge}")
        return None
    except nx.NodeNotFound as e:
        print(f"Node not found: {e}")
        return None

print("NetworkX shortest path function loaded")

NetworkX shortest path function loaded


In [10]:
# Helper functions for H3 high cell computation
def _find_ancestor_impl(cell: int, res: int) -> int:
    """Find ancestor of cell at given resolution."""
    if cell == 0:
        return 0
    cell_str = h3.int_to_str(cell)
    current_res = h3.get_resolution(cell_str)
    if res >= current_res:
        return cell
    return h3.str_to_int(h3.cell_to_parent(cell_str, res))

def find_lca(cell1: int, cell2: int) -> int:
    """Find lowest common ancestor of two H3 cells."""
    if cell1 == 0 or cell2 == 0:
        return 0
    
    cell1_str = h3.int_to_str(cell1)
    cell2_str = h3.int_to_str(cell2)
    
    res1 = h3.get_resolution(cell1_str)
    res2 = h3.get_resolution(cell2_str)
    
    # Start from the coarser resolution
    min_res = min(res1, res2)
    
    for res in range(min_res, -1, -1):
        ancestor1 = h3.str_to_int(h3.cell_to_parent(cell1_str, res))
        ancestor2 = h3.str_to_int(h3.cell_to_parent(cell2_str, res))
        if ancestor1 == ancestor2:
            return ancestor1
    
    return 0

def compute_high_cell(source_edge_id: int, destination_edge_id: int, edges_df: pd.DataFrame) -> tuple:
    """
    Compute the highest common ancestor H3 cell for source and destination edges.
    
    Returns:
        tuple: (highcell_int, highcell_resolution, source_cell_int, dest_cell_int)
    """
    # Get edge data
    source_edge = edges_df.loc[source_edge_id]
    dest_edge = edges_df.loc[destination_edge_id]
    
    # Get cells and resolutions
    s_cell = source_edge['incoming_cell']
    s_res = source_edge['lca_res']
    d_cell = dest_edge['incoming_cell']
    d_res = dest_edge['lca_res']
    
    # Find ancestors at lca_res
    source_cell = _find_ancestor_impl(s_cell, res=s_res)
    destination_cell = _find_ancestor_impl(d_cell, res=d_res)
    
    # Compute LCA
    highcell = find_lca(source_cell, destination_cell)
    
    if highcell == 0:
        highcell_res = -1
    else:
        highcell_res = h3.get_resolution(h3.int_to_str(highcell))
    
    return highcell, highcell_res, source_cell, destination_cell

print("H3 helper functions loaded")

H3 helper functions loaded


## Query Function

Function to query the C++ engine and parse the results

In [11]:
def query_shortest_path(source_edge, target_edge, dataset_config):
    """
    Query shortest path using C++ engine
    
    Args:
        source_edge: Source edge ID
        target_edge: Target edge ID
        dataset_config: Dataset configuration dictionary
    
    Returns:
        Dictionary with distance, shortcut_path, and expanded_path
    """
    cmd = [
        dataset_config['binary_path'],
        '--shortcuts', dataset_config['shortcuts_path'],
        '--edges', dataset_config['edges_path'],
        '--source', str(source_edge),
        '--target', str(target_edge)
    ]
    
    try:
        result = subprocess.run(cmd, capture_output=True, text=True, check=True)
        output = result.stdout
        
        # Parse output
        lines = output.strip().split('\n')
        
        distance = None
        shortcut_path = []
        expanded_path = []
        
        for line in lines:
            # Parse distance - handle formats like "Distance: 123.45" or "Total distance: 123.45"
            if 'istance:' in line:  # Matches both "Distance:" and "Total distance:"
                try:
                    # Extract the number after the colon
                    dist_str = line.split(':')[1].strip().split()[0]  # Get first token after colon
                    distance = float(dist_str)
                except (ValueError, IndexError):
                    pass
                    
            elif 'Shortcut path:' in line or ('Path:' in line and 'Expanded' not in line):
                # Extract path - format can be [1, 2, 3] or 1 -> 2 -> 3
                path_str = line.split(':', 1)[1].strip()
                
                # Check if it uses -> separator
                if '->' in path_str:
                    shortcut_path = [int(x.strip()) for x in path_str.split('->') if x.strip() and x.strip() != '...']
                else:
                    # Try bracket format [1, 2, 3]
                    path_str = path_str.strip('[]')
                    if path_str:
                        shortcut_path = [int(x.strip()) for x in path_str.split(',') if x.strip() and x.strip() != '...']
                        
            elif 'Expanded path:' in line or 'Expanded base edge path:' in line:
                # Extract expanded path
                path_str = line.split(':', 1)[1].strip()
                
                # Check if it uses -> separator
                if '->' in path_str:
                    expanded_path = [int(x.strip()) for x in path_str.split('->') if x.strip() and x.strip() != '...']
                else:
                    # Try bracket format [1, 2, 3]
                    path_str = path_str.strip('[]')
                    if path_str:
                        expanded_path = [int(x.strip()) for x in path_str.split(',') if x.strip() and x.strip() != '...']
        
        return {
            'distance': distance,
            'shortcut_path': shortcut_path,
            'expanded_path': expanded_path,
            'raw_output': output
        }
    
    except subprocess.CalledProcessError as e:
        print(f"Error running query: {e}")
        print(f"stdout: {e.stdout}")
        print(f"stderr: {e.stderr}")
        return None

## Visualization Function

Create an interactive map showing the route

In [12]:
def visualize_path(edge_ids, edge_lookup, title="Route", color="blue", highlight_endpoints=True, show_h3_cells=False):
    """
    Visualize a path on a Folium map
    
    Args:
        edge_ids: List of edge IDs forming the path
        edge_lookup: Dictionary mapping edge ID to edge data
        title: Map title
        color: Path color
        highlight_endpoints: Whether to highlight source and destination edges
        show_h3_cells: Whether to show H3 cells for source, destination, and highcell
    
    Returns:
        Folium Map object
    """
    if not edge_ids:
        print("No edges to visualize")
        return None
    
    # Collect all coordinates for centering
    all_coords = []
    
    # Parse geometries
    geometries = []
    for edge_id in edge_ids:
        if edge_id not in edge_lookup:
            print(f"Warning: Edge {edge_id} not found in edge data")
            continue
        
        edge_data = edge_lookup[edge_id]
        geom = wkt.loads(edge_data['geometry'])
        geometries.append((edge_id, geom, edge_data))
        
        # Extract coordinates
        coords = list(geom.coords)
        all_coords.extend(coords)
    
    if not all_coords:
        print("No valid geometries found")
        return None
    
    # Calculate center
    center_lat = sum(c[1] for c in all_coords) / len(all_coords)
    center_lon = sum(c[0] for c in all_coords) / len(all_coords)
    
    # Create map
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles='OpenStreetMap'
    )
    
    # Compute and display H3 cells if requested
    if show_h3_cells and len(edge_ids) >= 1:
        try:
            # Get source edge's incoming and outgoing cells
            source_edge_data = edge_lookup[edge_ids[0]]
            source_incoming_cell = source_edge_data['incoming_cell']
            source_outgoing_cell = source_edge_data['outgoing_cell']
            
            # Add incoming cell base
            if source_incoming_cell != 0:
                incoming_base_cell = h3.cell_to_parent(h3.int_to_str(source_incoming_cell), 0)
                incoming_boundary = h3.cell_to_boundary(incoming_base_cell)
                incoming_coords = [(lat, lon) for lat, lon in incoming_boundary]
                
                folium.Polygon(
                    incoming_coords,
                    color='blue',
                    weight=2,
                    fill=True,
                    fillColor='blue',
                    fillOpacity=0.05,
                    popup=f"Base H3 Cell (res 0) - Incoming Cell"
                ).add_to(m)
            
            # Add outgoing cell base
            if source_outgoing_cell != 0:
                outgoing_base_cell = h3.cell_to_parent(h3.int_to_str(source_outgoing_cell), 0)
                outgoing_boundary = h3.cell_to_boundary(outgoing_base_cell)
                outgoing_coords = [(lat, lon) for lat, lon in outgoing_boundary]
                
                folium.Polygon(
                    outgoing_coords,
                    color='purple',
                    weight=2,
                    fill=True,
                    fillColor='purple',
                    fillOpacity=0.05,
                    popup=f"Base H3 Cell (res 0) - Outgoing Cell"
                ).add_to(m)
        except Exception as e:
            print(f"Warning: Could not compute H3 cells: {e}")
    
    # Add path edges
    for idx, (edge_id, geom, edge_data) in enumerate(geometries):
        coords = [(lat, lon) for lon, lat in geom.coords]
        
        # Determine if this is source or destination edge
        is_source = (idx == 0) and highlight_endpoints
        is_destination = (idx == len(geometries) - 1) and highlight_endpoints
        
        # Set color and width based on position
        edge_color = color
        edge_weight = 4
        edge_opacity = 0.8
        
        if is_source:
            edge_color = 'green'
            edge_weight = 6
            edge_opacity = 0.9
        elif is_destination:
            edge_color = 'red'
            edge_weight = 6
            edge_opacity = 0.9
        
        # Create popup with edge info
        edge_type = ""
        if is_source:
            edge_type = " (SOURCE)"
        elif is_destination:
            edge_type = " (DESTINATION)"
            
        popup_text = f"""
        <b>Edge ID:</b> {edge_id}{edge_type}<br>
        <b>Length:</b> {edge_data.get('length', 'N/A'):.2f} m<br>
        <b>Highway:</b> {edge_data.get('highway', 'N/A')}<br>
        <b>Speed:</b> {edge_data.get('speed', 'N/A')} km/h
        """
        
        folium.PolyLine(
            coords,
            color=edge_color,
            weight=edge_weight,
            opacity=edge_opacity,
            popup=folium.Popup(popup_text, max_width=200)
        ).add_to(m)
    
    # Add start marker (green)
    if geometries:
        start_coords = list(geometries[0][1].coords)[0]
        folium.Marker(
            [start_coords[1], start_coords[0]],
            popup=f"Start (Edge {edge_ids[0]})",
            icon=folium.Icon(color='green', icon='play')
        ).add_to(m)
        
        # Add end marker (red)
        end_coords = list(geometries[-1][1].coords)[-1]
        folium.Marker(
            [end_coords[1], end_coords[0]],
            popup=f"End (Edge {edge_ids[-1]})",
            icon=folium.Icon(color='red', icon='stop')
        ).add_to(m)
    
    return m

## Example Query

Set your source and destination edge IDs here

In [33]:
# Set source and destination edge IDs
# For Somerset: try edges like 1593, 4835, 2310, 447, etc.
# For Burnaby: try edges like 30, 16904, 26838, etc.

#SOURCE_EDGE =415
#TARGET_EDGE =5253
SOURCE_EDGE =415
TARGET_EDGE = 9294
#407 → 1187 → 405 → 408 → 415 → 2770 → 5253 → 4388 → 7 → 5
#407 → 1187 → 405 → 408 → 415 → 547 → 544 → 538 → 483 → 2770
#Burnaby:  9219 → 34251 → 18292 → 18295 → 18298 → 18302 → 9214 → 9205 → 12199 → 1007 → 4768 → 25382 → 16394 → 24777 → 24778 → 24781 → 25366 → 5007 → 5005 → 5000
# ...   : 9219 → 34251 → 18292 → 18295 → 18298 → 18302 → 9214 → 9205 → 12199 → 1007 → 24723 → 13983 → 17860 → 13679 → 28879 → 25374 → 25377 → 13589 → 5029 → 4768
# ...   : 9219 → 34251 → 18292 → 18295 → 18298 → 18302 → 9214 → 9205 → 12199 → 1007 → 9377 → 24716 → 9363 → 28413 → 24717 → 24720 → 24721 → 4763 → 24723 → 13983
#9219 → 8818 → 9363 → 24723


print(f"Querying path from edge {SOURCE_EDGE} to edge {TARGET_EDGE}...")

Querying path from edge 415 to edge 9294...


In [34]:
# Query the path
result = query_shortest_path(SOURCE_EDGE, TARGET_EDGE, dataset)

if result:
    print("\n" + "="*60)
    print("QUERY RESULTS")
    print("="*60)
    
    # Handle distance (might be None)
    if result['distance'] is not None:
        print(f"Distance: {result['distance']:.2f} meters")
    else:
        print("Distance: Not available")
    
    print(f"\nShortcut Path ({len(result['shortcut_path'])} edges):")
    print(" → ".join(map(str, result['shortcut_path'])))
    print(f"\nExpanded Path ({len(result['expanded_path'])} base edges):")
    print(" → ".join(map(str, result['expanded_path'])))
    print("\n" + "="*60)
    print("\nRaw C++ Output:")
    print(result['raw_output'])
else:
    print("Query failed!")


QUERY RESULTS
Distance: Not available

Shortcut Path (3 edges):
415 → 11026 → 9294

Expanded Path (69 base edges):
415 → 32877 → 17260 → 11546 → 25033 → 25034 → 17235 → 17226 → 17236 → 17240 → 17244 → 25035 → 25039 → 25041 → 11181 → 25045 → 18133 → 33598 → 33601 → 31792 → 24251 → 10993 → 24272 → 11493 → 25945 → 3759 → 25889 → 25893 → 26581 → 28446 → 20288 → 20304 → 25067 → 28448 → 5595 → 11573 → 31157 → 31155 → 21963 → 30102 → 29184 → 25065 → 25057 → 25063 → 11772 → 25156 → 19029 → 9287 → 9289 → 9602 → 19051 → 10963 → 17868 → 11026 → 30844 → 17655 → 157 → 151 → 1825 → 24398 → 24400 → 24405 → 25318 → 155 → 9290 → 23119 → 23120 → 23124 → 9294


Raw C++ Output:
Loading shortcuts from ../../spark-shortest-path/output/Burnaby_shortcuts_final...
Loading edge metadata from ../../osm-to-road-network/data/output/Burnaby_driving_simplified_edges_with_h3.csv...
Query 415 -> 9294
  Distance (including destination edge): 789.884
  Destination edge cost: 684.05
  Shortcut path length: 3 edges
  Sho

## Visualize Shortcut Path

## Compare with NetworkX

Run NetworkX shortest path and compare results

In [35]:
# Compare with NetworkX
if result:
    print("\n" + "="*60)
    print("NETWORKX COMPARISON")
    print("="*60)
    
    nx_result = networkx_shortest_path(SOURCE_EDGE, TARGET_EDGE, G)
    
    if nx_result:
        print(f"NetworkX Distance: {nx_result['distance']:.2f} meters")
        print(f"NetworkX Path ({len(nx_result['path'])} edges):")
        print(" → ".join(map(str, nx_result['path'])))
        
        # Compare with your algorithm
        print("\n" + "-"*60)
        print("COMPARISON")
        print("-"*60)
        
        if result['distance'] is not None:
            your_distance = result['distance']
            nx_distance = nx_result['distance']
            print(f"Your algorithm distance: {your_distance:.2f} meters")
            print(f"NetworkX distance:      {nx_distance:.2f} meters")
            print(f"Difference:             {abs(your_distance - nx_distance):.2f} meters")
            
            if abs(your_distance - nx_distance) < 0.01:
                print("✓ Distances match!")
            else:
                print("✗ Distances differ!")
        
        # Compare expanded paths
        your_path = result['expanded_path']
        nx_path = nx_result['path']
        
        print(f"\nYour expanded path length: {len(your_path)} edges")
        print(f"NetworkX path length:      {len(nx_path)} edges")
        
        if your_path == nx_path:
            print("✓ Paths are identical!")
        else:
            print("✗ Paths differ!")
            print("\nYour path:    ", " → ".join(map(str, your_path)))
            print("NetworkX path:", " → ".join(map(str, nx_path)))
    else:
        print("NetworkX could not find a path")
else:
    print("No results to compare")


NETWORKX COMPARISON
NetworkX Distance: 381.00 meters
NetworkX Path (69 edges):
415 → 32877 → 17260 → 11546 → 25033 → 25034 → 17235 → 17226 → 17236 → 17240 → 17244 → 25035 → 25039 → 25041 → 11181 → 25045 → 18133 → 33598 → 33601 → 31792 → 24251 → 10993 → 24272 → 11493 → 25945 → 3759 → 25889 → 25893 → 26581 → 28446 → 20288 → 20304 → 25067 → 28448 → 5595 → 11573 → 31157 → 31155 → 21963 → 30102 → 29184 → 25065 → 25057 → 25063 → 11772 → 25156 → 19029 → 9287 → 9289 → 9602 → 19051 → 10963 → 17868 → 11026 → 30844 → 17655 → 157 → 151 → 1825 → 24398 → 24400 → 24405 → 25318 → 155 → 9290 → 23119 → 23120 → 23124 → 9294

------------------------------------------------------------
COMPARISON
------------------------------------------------------------

Your expanded path length: 69 edges
NetworkX path length:      69 edges
✓ Paths are identical!


In [36]:
if result and result['shortcut_path']:
    shortcut_map = visualize_path(
        result['shortcut_path'],
        edge_lookup,
        title="Shortcut Path",
        color="blue",
        show_h3_cells=True
    )
    display(shortcut_map)
else:
    print("No shortcut path to visualize")

## Visualize Expanded Path

In [37]:
if result and result['expanded_path']:
    expanded_map = visualize_path(
        result['expanded_path'],
        edge_lookup,
        title="Expanded Base Edge Path",
        color="red",
        show_h3_cells=True
    )
    display(expanded_map)
else:
    print("No expanded path to visualize")

## Visualize NetworkX Path

Display the NetworkX shortest path (ground truth) for comparison

In [38]:
# Visualize NetworkX shortest path
if 'nx_result' in locals() and nx_result and nx_result['path']:
    networkx_map = visualize_path(
        nx_result['path'],
        edge_lookup,
        title="NetworkX Shortest Path (Ground Truth)",
        color="orange",
        show_h3_cells=False
    )
    display(networkx_map)
else:
    print("No NetworkX path to visualize - run the comparison cell first")

## Compare Both Paths

Overlay both paths to see the difference

In [19]:
if result and result['shortcut_path'] and result['expanded_path']:
    # Create combined map
    all_edges = result['expanded_path']
    
    # Calculate center
    all_coords = []
    for edge_id in all_edges:
        if edge_id in edge_lookup:
            geom = wkt.loads(edge_lookup[edge_id]['geometry'])
            all_coords.extend(list(geom.coords))
    
    center_lat = sum(c[1] for c in all_coords) / len(all_coords)
    center_lon = sum(c[0] for c in all_coords) / len(all_coords)
    
    m = folium.Map(
        location=[center_lat, center_lon],
        zoom_start=13,
        tiles='OpenStreetMap'
    )
    
    # Add expanded path (red, thinner)
    for edge_id in result['expanded_path']:
        if edge_id in edge_lookup:
            edge_data = edge_lookup[edge_id]
            geom = wkt.loads(edge_data['geometry'])
            coords = [(lat, lon) for lon, lat in geom.coords]
            folium.PolyLine(
                coords,
                color='red',
                weight=3,
                opacity=0.6,
                popup=f"Expanded Edge {edge_id}"
            ).add_to(m)
    
    # Add shortcut path (blue, thicker, on top)
    for edge_id in result['shortcut_path']:
        if edge_id in edge_lookup:
            edge_data = edge_lookup[edge_id]
            geom = wkt.loads(edge_data['geometry'])
            coords = [(lat, lon) for lon, lat in geom.coords]
            folium.PolyLine(
                coords,
                color='blue',
                weight=5,
                opacity=0.8,
                popup=f"Shortcut Edge {edge_id}"
            ).add_to(m)
    
    # Add markers
    start_geom = wkt.loads(edge_lookup[result['expanded_path'][0]]['geometry'])
    start_coords = list(start_geom.coords)[0]
    folium.Marker(
        [start_coords[1], start_coords[0]],
        popup=f"Start (Edge {result['expanded_path'][0]})",
        icon=folium.Icon(color='green', icon='play')
    ).add_to(m)
    
    end_geom = wkt.loads(edge_lookup[result['expanded_path'][-1]]['geometry'])
    end_coords = list(end_geom.coords)[-1]
    folium.Marker(
        [end_coords[1], end_coords[0]],
        popup=f"End (Edge {result['expanded_path'][-1]})",
        icon=folium.Icon(color='red', icon='stop')
    ).add_to(m)
    
    # Add legend
    legend_html = '''
    <div style="position: fixed; 
                bottom: 50px; left: 50px; width: 200px; height: 90px; 
                background-color: white; border:2px solid grey; z-index:9999; 
                font-size:14px; padding: 10px">
    <b>Legend</b><br>
    <i style="background: blue; width: 30px; height: 3px; display: inline-block;"></i> Shortcut Path<br>
    <i style="background: red; width: 30px; height: 3px; display: inline-block;"></i> Expanded Path
    </div>
    '''
    m.get_root().html.add_child(folium.Element(legend_html))
    
    display(m)
else:
    print("Cannot create comparison - missing path data")

## Path Analysis

Analyze the differences between shortcut and expanded paths

In [20]:
if result:
    shortcut_count = len(result['shortcut_path'])
    expanded_count = len(result['expanded_path'])
    
    print("\n" + "="*60)
    print("PATH ANALYSIS")
    print("="*60)
    print(f"Shortcut edges: {shortcut_count}")
    print(f"Expanded base edges: {expanded_count}")
    print(f"Compression ratio: {expanded_count/shortcut_count:.2f}x")
    
    if result['distance'] is not None:
        print(f"Total distance: {result['distance']:.2f} meters")
    else:
        print("Total distance: Not available")
    
    # Check which shortcut edges were expanded
    print("\nShortcut expansion details:")
    for i, shortcut_edge in enumerate(result['shortcut_path']):
        if shortcut_edge in result['expanded_path']:
            print(f"  Edge {shortcut_edge}: Real edge (no expansion)")
        else:
            print(f"  Edge {shortcut_edge}: Shortcut (expanded to multiple edges)")


PATH ANALYSIS
Shortcut edges: 2
Expanded base edges: 54
Compression ratio: 27.00x
Total distance: Not available

Shortcut expansion details:
  Edge 415: Real edge (no expansion)
  Edge 11026: Real edge (no expansion)


## Interactive Query

Query multiple paths in a loop

In [21]:
# Example: Query multiple paths
test_queries = [
    (2311, 447),   # Somerset example
    (1593, 4835),  # Somerset example
    # Add more (source, target) pairs here
]

for source, target in test_queries:
    print(f"\n{'='*60}")
    print(f"Query: {source} → {target}")
    print('='*60)
    
    result = query_shortest_path(source, target, dataset)
    
    if result:
        if result['distance'] is not None:
            print(f"Distance: {result['distance']:.2f} m")
        else:
            print("Distance: Not available")
        print(f"Shortcut path: {' → '.join(map(str, result['shortcut_path']))}")
        print(f"Expanded path: {' → '.join(map(str, result['expanded_path']))}")
    else:
        print("Query failed!")


Query: 2311 → 447
Distance: Not available
Shortcut path: 2311 → 11170 → 1573 → 1566 → 20386 → 439 → 447
Expanded path: 2311 → 11170 → 1573 → 1570 → 1566 → 32828 → 26947 → 26935 → 26933 → 27592 → 18433 → 26671 → 2524 → 33580 → 24682 → 20868 → 24501 → 24499 → 19938 → 1564 → 24612 → 24611 → 27679 → 27682 → 20380 → 20386 → 20388 → 24608 → 24606 → 31384 → 28442 → 28444 → 24543 → 24545 → 22849 → 22586 → 27674 → 22584 → 16466 → 16465 → 25008 → 25010 → 17796 → 4546 → 17403 → 10736 → 4517 → 4513 → 25076 → 10455 → 25315 → 16442 → 15354 → 4468 → 15611 → 15330 → 487 → 16445 → 670 → 33998 → 33573 → 15337 → 33564 → 644 → 15552 → 33029 → 14807 → 21680 → 11838 → 34000 → 22365 → 25242 → 11476 → 23983 → 11670 → 20690 → 20694 → 14891 → 28356 → 15008 → 673 → 15616 → 15067 → 28360 → 23797 → 15039 → 693 → 15142 → 17822 → 23408 → 702 → 3742 → 439 → 442 → 447

Query: 1593 → 4835
Distance: Not available
Shortcut path: 1593 → 34486 → 24508 → 8504 → 4835
Expanded path: 1593 → 6419 → 1591 → 34486 → 1588 → 27191 

In [22]:
import pandas as pd

district = "Somerset" #"Burnaby"
# Update this path if you want to work with a different edge metadata snapshot.
edges_PATH = f"../../spark-shortest-path/data/{district}_driving_simplified_edges_with_h3.csv"

# Load the edge table and retain it with the edge id as index for fast lookups.
edges = pd.read_csv(edges_PATH)
edges_df = edges.set_index('id')
print(f"Loaded {len(edges):,} all edges")
edges.head()

Loaded 5,900 all edges


,source,target,length,maxspeed,geometry,highway,cost,incoming_cell,outgoing_cell,lca_res,id
0,167790704,167862530,80.287,60.0,LINESTRING (-84.60797882080078 37.091793060302...,secondary,4.817220,645224977384028320,645224977383611141,8,0
1,167790704,167790717,102.097,60.0,LINESTRING (-84.60797882080078 37.091793060302...,secondary,6.125820,645224977383614665,645224977383611141,10,1
2,167790704,167825885,323.104,50.0,LINESTRING (-84.60797882080078 37.091793060302...,tertiary,23.263488,645224977384658840,645224977383611141,8,2
3,167790717,167790719,111.432,60.0,LINESTRING (-84.60697937011719 37.091552734375...,secondary,6.685920,645224977383653531,645224977383614665,9,3
4,167790717,167862553,84.317,30.0,LINESTRING (-84.60697937011719 37.091552734375...,residential,10.118040,645224977383600429,645224977383614665,10,4


In [23]:
edges[edges["id"]==SOURCE_EDGE]

,source,target,length,maxspeed,geometry,highway,cost,incoming_cell,outgoing_cell,lca_res,id
415,167829988,167965084,168.82,30.0,LINESTRING (-84.59919738769531 37.091606140136...,residential,20.2584,645224977387271749,645224977388213771,8,415


In [24]:
edges[edges["id"]==TARGET_EDGE]

,source,target,length,maxspeed,geometry,highway,cost,incoming_cell,outgoing_cell,lca_res,id


In [25]:
PARQUET_PATH = f"../../spark-shortest-path/output/{district}_shortcuts_final"

# Load shortcuts and normalise schema for downstream computations.
df = pd.read_parquet(PARQUET_PATH)

# Ensure deterministic types for the routing algorithm.
df['incoming_edge'] = df['incoming_edge'].astype(int)
df['outgoing_edge'] = df['outgoing_edge'].astype(int)
df['via_edge'] = df['via_edge'].fillna(0).astype(int)
df['inside'] = df['inside'].astype(int)
df['cost'] = df['cost'].astype(float)

print(f"Loaded {len(df):,} shortcut edges")
df.head()

Loaded 213,493 shortcut edges


,incoming_edge,outgoing_edge,cost,via_edge,inside,cell
0,2310,447,31.018371,2311,0,0
1,2310,3053,42.085436,2311,0,0
2,2310,3365,43.738064,2311,0,0
3,2310,3354,45.669436,2311,0,0
4,2310,4469,195.984652,2311,0,0


In [26]:
import pandas as pd

district = "Somerset" #"Burnaby"
# Update this path if you want to work with a different edge metadata snapshot.
edges_graph_PATH = f"../../osm-to-road-network/data/output/{district}_driving_edge_graph.csv"
# Load the edge table and retain it with the edge id as index for fast lookups.
edges_graph = pd.read_csv(edges_graph_PATH)
print(f"Loaded {len(edges_graph):,} all edges")
edges_graph.head()

Loaded 16,301 all edges


,incoming_edge,outgoing_edge
0,1516,3778
1,1166,669
2,5203,5208
3,937,5
4,965,2345


In [27]:
edges_graph[edges_graph["incoming_edge"]==SOURCE_EDGE]

,incoming_edge,outgoing_edge
14282,415,2199
15699,415,2200
16225,415,2201


In [28]:
edges_graph[edges_graph["incoming_edge"]==2309]

,incoming_edge,outgoing_edge
2326,2309,445
6471,2309,447
10851,2309,446


In [29]:
PARQUET_PATH = f"../../spark-shortest-path/output/{district}_shortcuts_final"

# Load shortcuts and normalise schema for downstream computations.
df = pd.read_parquet(PARQUET_PATH)

# Ensure deterministic types for the routing algorithm.
df['incoming_edge'] = df['incoming_edge'].astype(int)
df['outgoing_edge'] = df['outgoing_edge'].astype(int)
df['via_edge'] = df['via_edge'].fillna(0).astype(int)
df['inside'] = df['inside'].astype(int)
df['cost'] = df['cost'].astype(float)

print(f"Loaded {len(df):,} shortcut edges")
df.head()

Loaded 213,493 shortcut edges


,incoming_edge,outgoing_edge,cost,via_edge,inside,cell
0,2310,447,31.018371,2311,0,0
1,2310,3053,42.085436,2311,0,0
2,2310,3365,43.738064,2311,0,0
3,2310,3354,45.669436,2311,0,0
4,2310,4469,195.984652,2311,0,0


In [30]:
# Diagnostic: Check graph connectivity for the query
print(f"Checking connectivity from {SOURCE_EDGE} to {TARGET_EDGE}:")
print(f"\nDirect edge exists? {G.has_edge(SOURCE_EDGE, TARGET_EDGE)}")

if G.has_node(SOURCE_EDGE):
    neighbors = list(G.neighbors(SOURCE_EDGE))
    print(f"\nEdges reachable from {SOURCE_EDGE}: {neighbors[:10]}")  # Show first 10
    print(f"Total neighbors: {len(neighbors)}")
else:
    print(f"\n{SOURCE_EDGE} not in graph!")

if G.has_node(TARGET_EDGE):
    predecessors = list(G.predecessors(TARGET_EDGE))
    print(f"\nEdges leading to {TARGET_EDGE}: {predecessors[:10]}")  # Show first 10
    print(f"Total predecessors: {len(predecessors)}")
else:
    print(f"\n{TARGET_EDGE} not in graph!")

Checking connectivity from 415 to 11026:

Direct edge exists? False

Edges reachable from 415: [32877, 32878, 32876, 32879]
Total neighbors: 4

Edges leading to 11026: [17868]
Total predecessors: 1


In [31]:
# Check what NetworkX actually returns
print(f"Query: {SOURCE_EDGE} → {TARGET_EDGE}")

if nx_result:
    print(f"\nNetworkX found a path!")
    print(f"Path length: {len(nx_result['path'])} edges")
    print(f"Full path: {nx_result['path']}")
    print(f"Distance: {nx_result['distance']:.2f}")
    
    # Check if path is actually just 2 edges or more
    if len(nx_result['path']) == 2:
        print("\n⚠️ WARNING: Path only contains source and target!")
        print("This means NetworkX found a DIRECT connection.")
        print(f"Checking if direct edge exists: {G.has_edge(SOURCE_EDGE, TARGET_EDGE)}")
    else:
        print(f"\n✓ Path contains {len(nx_result['path']) - 2} intermediate edges")
        print(f"Intermediate edges: {nx_result['path'][1:-1]}")
else:
    print("NetworkX did not find a path")

Query: 415 → 11026

NetworkX found a path!
Path length: 54 edges
Full path: [415, 32877, 17260, 11546, 25033, 25034, 17235, 17226, 17236, 17240, 17244, 25035, 25039, 25041, 11181, 25045, 18133, 33598, 33601, 31792, 24251, 10993, 24272, 11493, 25945, 3759, 25889, 25893, 26581, 28446, 20288, 20304, 25067, 28448, 5595, 11573, 31157, 31155, 21963, 30102, 29184, 25065, 25057, 25063, 11772, 25156, 19029, 9287, 9289, 9602, 19051, 10963, 17868, 11026]
Distance: 222.57

✓ Path contains 52 intermediate edges
Intermediate edges: [32877, 17260, 11546, 25033, 25034, 17235, 17226, 17236, 17240, 17244, 25035, 25039, 25041, 11181, 25045, 18133, 33598, 33601, 31792, 24251, 10993, 24272, 11493, 25945, 3759, 25889, 25893, 26581, 28446, 20288, 20304, 25067, 28448, 5595, 11573, 31157, 31155, 21963, 30102, 29184, 25065, 25057, 25063, 11772, 25156, 19029, 9287, 9289, 9602, 19051, 10963, 17868]


In [32]:
# Diagnostic: Check if all edges in nx_result are in edge_lookup
if nx_result and nx_result['path']:
    print(f"Checking edge_lookup for {len(nx_result['path'])} edges:")
    missing = []
    for edge_id in nx_result['path']:
        if edge_id not in edge_lookup:
            missing.append(edge_id)
    
    if missing:
        print(f"⚠️ WARNING: {len(missing)} edges NOT found in edge_lookup!")
        print(f"Missing edges: {missing[:10]}")  # Show first 10
    else:
        print(f"✓ All {len(nx_result['path'])} edges found in edge_lookup")

Checking edge_lookup for 54 edges:
✓ All 54 edges found in edge_lookup
